In [4]:
import requests


In [2]:
import getpass
username = input('Email:')
password = getpass.getpass('Password:')


Email:kfronczak@gmail.com
Password:········


In [5]:
'''Gets Authentication Token'''
url     = 'https://prod.immedia-semi.com/login'
data    = '{ "password" : "'+password+'", "client_specifier" : "iPhone 9.2 | 2.2 | 222", "email" : "'+username+'" }'
headers = {'Host': 'prod.immedia-semi.com',
           'Content-Type': 'application/json',
          }

response = requests.post(url, headers=headers, data=data)

authtoken = response.json()['authtoken']['authtoken']

In [6]:
print(response.json())

{'authtoken': {'authtoken': '5Zqe5Pt0h39ajo3oZeltug', 'message': 'auth'}, 'networks': {'2535': {'name': 'Varys', 'onboarded': True}}, 'region': {'prod': 'United States'}}


In [9]:
'''Gets Network ID and Account ID'''
url     = 'https://prod.immedia-semi.com/networks'
headers = {'Host': 'prod.immedia-semi.com',
           'TOKEN_AUTH': authtoken,
          }

response = requests.get(url, headers=headers)

network_id = response.json()['networks'][0]['id']
account_id = response.json()['networks'][0]['account_id']


In [10]:
print(response.json())

{'networks': [{'arm_string': 'Disarmed', 'lv_mode': 'relay', 'locale': '', 'ping_interval': 60, 'dst': True, 'sync_module_error': False, 'created_at': '2016-01-08T22:54:11+00:00', 'storage_total': 0, 'lfr_channel': 0, 'autoarm_geo_enable': False, 'id': 2535, 'busy': False, 'storage_used': 0, 'encryption_key': None, 'video_destination': 'server', 'armed': False, 'video_history_count': 4000, 'feature_plan_id': None, 'name': 'Varys', 'updated_at': '2017-01-22T00:36:55+00:00', 'autoarm_time_enable': False, 'network_key': None, 'network_origin': 'normal', 'camera_error': False, 'time_zone': 'America/New_York', 'video_count': 0, 'description': '', 'account_id': 2463}], 'summary': {'2535': {'name': 'Varys', 'onboarded': True}}}


In [23]:
'''Checks to see if sync module is online'''
url     = 'https://prod.immedia-semi.com/network/'+str(network_id)+'/syncmodules'
headers = {'Host': 'prod.immedia-semi.com',
           'TOKEN_AUTH': authtoken,
          }

response = requests.get(url, headers=headers)

status   = response.json()['syncmodule']['status']

if status == 'online':
    print('Sync Module Online!')
else:
    print('ERROR: Sync Module is '+status)

print(response.json())

Sync Module Online!
{'syncmodule': {'onboarded': True, 'ip_address': '67.244.152.156', 'server': 'i-03767a4b62de7beb3', 'last_wifi_alert': None, 'wifi_alert_count': 0, 'offline_alert_count': 0, 'os_version': '1.1.0', 'status': 'online', 'account_id': 2463, 'serial': '270002045', 'created_at': '2016-01-08T22:56:47+00:00', 'mac_address': '00:03:7f:11:4f:ba', 'id': 1487, 'name': 'Blink SM', 'fw_version': '1.55', 'lfr_frequency': None, 'feature_plan_id': None, 'last_offline_alert': None, 'network_id': 2535, 'updated_at': '2017-01-19T03:34:51+00:00', 'last_hb': '2017-01-21T03:49:36+00:00'}}


In [20]:
'''Arm or Disarm Module'''
status  = 'disarm'
url     =  'https://prod.immedia-semi.com/network/'+str(network_id)+'/'+status
headers = {'Host': 'prod.immedia-semi.com',
           'TOKEN_AUTH': authtoken,
          }
response = requests.post(url, headers=headers)


In [22]:
print(response.json())

{'sequence': None, 'camera_id': None, 'lfr_ack': None, 'stage_cs_sent': None, 'stage_vs': None, 'stage_dev': None, 'server': None, 'stage_rest': '2017-01-21T16:55:45+00:00', 'network_id': 2535, 'sm_ack': None, 'siren_id': None, 'firmware_id': None, 'stage_is': None, 'debug': '', 'by_whom': '', 'sync_module_id': 1487, 'stage_cs_db': None, 'duration': None, 'diagnostic': False, 'transaction': 'qnR3whVmznpNs2R1', 'parent_command_id': None, 'state_condition': 'new', 'state_stage': 'rest', 'stage_lv': None, 'updated_at': '2017-01-21T16:55:45+00:00', 'id': 68307295, 'command': 'disarm', 'account_id': 2463, 'attempts': 0, 'created_at': '2017-01-21T16:55:45+00:00', 'stage_sm': None, 'execute_time': '2017-01-21T16:55:45+00:00', 'player_transaction': 'y-BglXz-HIB_PM9-'}


In [11]:
'''Get Summary'''
url     = 'https://prod.immedia-semi.com/homescreen'
headers = {'Host': 'prod.immedia-semi.com',
           'TOKEN_AUTH': authtoken,
          }

response = requests.get(url, headers=headers)

jData  = response.json()['devices']

cameras = {}

for element in jData:
            if 'device_type' in element.keys():
                if element['device_type'] == 'camera':
                    cameras[element['name']] = element
           
    
print(cameras)

{'Living Room': {'active': 'disabled', 'battery': 3, 'warning': 0, 'error_msg': '', 'updated_at': '2017-01-23T05:36:15+00:00', 'name': 'Living Room', 'device_id': 3875, 'errors': 0, 'temp': 67, 'armed': False, 'enabled': False, 'thumbnail': '/media/production/account/2463/network/2535/camera/3875/clip_4Ndek4vZ_2017_01_23__05_36AM', 'device_type': 'camera', 'status': 'done', 'wifi_strength': 5, 'lfr_strength': 4, 'notifications': 1}}


In [51]:
'''Get list of events and save most recent image'''
url     = 'https://prod.immedia-semi.com/events/network/'+str(network_id)
headers = {'Host': 'prod.immedia-semi.com',
           'TOKEN_AUTH': authtoken,
          }

response = requests.get(url, headers=headers)

jData = response.json()['event']

for i in range(0, len(jData)):
    if 'video_url' in jData[i]:
        last_event = jData[i]['video_url']
        break

if last_event:
    video_url = 'https://prod.immedia-semi.com/'+last_event
    image_url = 'https://prod.immedia-semi.com/'+last_event[:-3]+'jpg'
    
    #with open('thumb.jpg', 'wb') as handle:
    #    response = requests.get(image_url, headers=headers)
    #    
    ##    if not response.ok:
     #       print('UH-OH!!')
     #   
     #   for block in response.iter_content(1024):
     #       handle.write(block)

In [52]:
print(jData)

[{'status': None, 'sync_module_id': 1487, 'syncmodule': 1487, 'notified': False, 'duration': None, 'camera_id': None, 'account_id': 2463, 'network_id': None, 'updated_at': '2017-01-21T16:55:45+00:00', 'id': 260103702, 'siren': None, 'siren_id': None, 'camera': None, 'command_id': None, 'created_at': '2017-01-21T16:55:45+00:00', 'account': 2463, 'type': 'disarmed'}, {'status': None, 'sync_module_id': 1487, 'syncmodule': 1487, 'notified': False, 'duration': None, 'camera_id': None, 'account_id': 2463, 'network_id': None, 'updated_at': '2017-01-21T15:16:58+00:00', 'id': 260005900, 'siren': None, 'siren_id': None, 'camera': None, 'command_id': None, 'created_at': '2017-01-21T15:16:58+00:00', 'account': 2463, 'type': 'first_boot'}, {'status': None, 'sync_module_id': 1487, 'syncmodule': 1487, 'notified': False, 'duration': None, 'camera_id': None, 'account_id': 2463, 'network_id': None, 'updated_at': '2017-01-21T15:16:44+00:00', 'id': 260005658, 'siren': None, 'siren_id': None, 'camera': Non

In [42]:
'''New thumbnail'''
url     = 'https://prod.immedia-semi.com/network/'+str(network_id)+'/camera/'+str(cameras['Living Room']['id'])+'/thumbnail'
headers = {'Host': 'prod.immedia-semi.com',
           'TOKEN_AUTH': authtoken,
          }

response = requests.post(url, headers=headers)
print(response.json())

{'code': 500, 'message': 'Camera not found'}


In [40]:
print(url)

https://prod.immedia-semi.com/network/2535/camera/3875/thumbnail


In [50]:
x= {'test':123, 'name':'req'}

if 'test' in x.keys() and x['testd'] == 123:
    print(x['name'])
else:
    print('Oops')

KeyError: 'testd'

In [57]:
print(url.split("/"))

['https:', '', 'prod.immedia-semi.com', 'events', 'network', '2535']


In [60]:
x = {'123':'abc', '456':'def'}
for key, value in x.items():
    print(value)

abc
def


In [8]:
(key, value), = response.json()['region'].items()
print(value)


United States
